In [1]:
import pandas as pd
import os
test_data = pd.read_json('/Users/mrinaalarora/Downloads/Spotify Extended Streaming History/Streaming_History_Video_2023-2024.json')
test_data.tail()

,ts,username,platform,ms_played,conn_country,ip_addr_decrypted,user_agent_decrypted,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,...,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
181,2024-04-10T03:44:14Z,31nr24ptwueq6pagibzbnwmugamm,ios,1012,US,70.167.111.242,unknown,Open Arms (feat. Travis Scott),SZA,SOS,...,None,None,None,unknown,endplay,False,True,NaN,0,False
182,2024-04-10T03:44:14Z,31nr24ptwueq6pagibzbnwmugamm,ios,610,US,70.167.111.242,unknown,Saturnine,Justice,Saturnine,...,None,None,None,unknown,endplay,False,True,NaN,0,False
183,2024-04-10T03:44:15Z,31nr24ptwueq6pagibzbnwmugamm,ios,461,US,70.167.111.242,unknown,16,Baby Keem,The Melodic Blue,...,None,None,None,unknown,endplay,False,True,NaN,0,False
184,2024-04-10T03:44:41Z,31nr24ptwueq6pagibzbnwmugamm,ios,305,US,70.167.111.242,unknown,"I miss you, I’m sorry",Gracie Abrams,minor,...,None,None,None,unknown,endplay,False,True,NaN,0,False
185,2024-04-10T15:34:47Z,31nr24ptwueq6pagibzbnwmugamm,ios,3286,US,172.56.208.16,unknown,None,None,None,...,What happened in Vegas,Hamza,spotify:episode:7ujU3RntadeivhOV8dRPBa,trackdone,endplay,True,True,NaN,0,False
